# Hudi + Lance Demo: Intelligent Recruitment Platform
**(Hybrid Search + Analytics on the Lakehouse)**

One table. Three query patterns. Zero data copying.

1. Load real job postings from HuggingFace
2. Ingest into a Hudi table with Lance vector embeddings
3. **Vector Search** — match a resume by meaning
4. **Hybrid Search** — add business constraints (SQL + vectors)
5. **Analytics** — executive dashboard on the same data

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from datasets import load_dataset
import shutil, os, sys
import pandas as pd
import matplotlib.pyplot as plt
from sentence_transformers import SentenceTransformer

TABLE_PATH = "/tmp/hudi_recruiting_lake"
TABLE_NAME = "job_market"
EMBEDDING_MODEL = "all-MiniLM-L6-v2"

# Spark/Lance emit log lines with surrogate characters that break
# Jupyter's JSON serializer. Wrap both streams to sanitize them.
class _SafeStream:
    def __init__(self, stream):
        self._stream = stream
    def write(self, s):
        return self._stream.write(s.encode("utf-8", errors="replace").decode("utf-8"))
    def flush(self):
        return self._stream.flush()
    def __getattr__(self, name):
        return getattr(self._stream, name)

sys.stdout = _SafeStream(sys.stdout)
sys.stderr = _SafeStream(sys.stderr)

## 1. Start Spark with Hudi + Lance

In [ ]:
spark = (SparkSession.builder.appName("Recruiting-Lakehouse")
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer")
    .config("spark.sql.extensions", "org.apache.spark.sql.hudi.HoodieSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.hudi.catalog.HoodieCatalog")
    .config("spark.ui.showConsoleProgress", "false")
    .getOrCreate())

spark.sparkContext.setLogLevel("ERROR")

print(f"\u2713 Spark {spark.version} ready with Hudi extensions.")

## 2. Load Real Job Postings from HuggingFace

~3k data science job descriptions from [nathansutton/data-science-job-descriptions](https://huggingface.co/datasets/nathansutton/data-science-job-descriptions).

In [ ]:
ds = load_dataset("nathansutton/data-science-job-descriptions", split="train")

jobs_data = []
for i, row in enumerate(ds):
    jobs_data.append({
        "job_id": f"job_{i:04d}",
        "company": row["company"],
        "title": row["title"],
        "job_description": row["job_description"],
        "text_for_vector": f"{row['title']} {row['job_description']}"
    })

companies = set(r["company"] for r in jobs_data)
print(f"\u2713 Loaded {len(jobs_data)} job postings from {len(companies)} companies.\n")
pd.DataFrame(jobs_data, columns=["job_id", "company", "title"]).head(10)

## 3. Embed & Ingest into the Lakehouse

We embed every job description into a 384-dim vector, then write structured fields **and** embeddings into a single Hudi table using the Lance file format.

In [ ]:
model = SentenceTransformer(EMBEDDING_MODEL)
embeddings = model.encode([r["text_for_vector"] for r in jobs_data], show_progress_bar=True)

for i, row in enumerate(jobs_data):
    row["embedding"] = embeddings[i].tolist()

print(f"\u2713 Generated {len(embeddings)} embeddings (dim={len(embeddings[0])}).\n")

preview = pd.DataFrame(jobs_data, columns=["job_id", "title", "embedding"])
preview["embedding"] = preview["embedding"].apply(lambda v: str(v[:4])[:-1] + ", ...]")
preview.head(10)

In [ ]:
schema = StructType([
    StructField("job_id", StringType(), False),
    StructField("company", StringType(), False),
    StructField("title", StringType(), False),
    StructField("job_description", StringType(), False),
    StructField("text_for_vector", StringType(), False),
    StructField("embedding", ArrayType(FloatType()), False),
])

if os.path.exists(TABLE_PATH):
    shutil.rmtree(TABLE_PATH)

df = spark.createDataFrame(jobs_data, schema=schema)

hudi_options = {
    "hoodie.table.name": TABLE_NAME,
    "hoodie.datasource.write.recordkey.field": "job_id",
    "hoodie.datasource.write.partitionpath.field": "company",
    "hoodie.datasource.write.table.type": "COPY_ON_WRITE",
    "hoodie.datasource.write.operation": "bulk_insert",
    "hoodie.table.base.file.format": "lance",
    "hoodie.write.record.merge.custom.implementation.classes": "org.apache.hudi.DefaultSparkRecordMerger"
}

df.write.format("hudi").options(**hudi_options).mode("overwrite").save(TABLE_PATH)
print(f"\u2713 Ingested {len(jobs_data)} jobs into Hudi table at {TABLE_PATH}")

---
## 4. Vector Search — "Upload" a Resume

A candidate uploads their resume. It never says *"Senior Data Scientist"* — it describes skills like *"deployed LLMs"* and *"Scikit-Learn."* Can the system find the right jobs anyway?

In [ ]:
resume_text = """
EXPERIENCE:
- 5 years building Machine Learning models using Python and Scikit-Learn.
- Deployed Large Language Models (LLMs) to production.
- Strong background in backend engineering and API design.
"""

print(f"\U0001F4C4 Resume Uploaded:\n{resume_text.strip()}")

In [ ]:
resume_vector = model.encode([resume_text])[0].tolist()
spark.createDataFrame([(resume_vector,)], ["q_vec"]).createOrReplaceTempView("query_input")

matches = spark.sql(f"""
    SELECT title, company, (1 - _distance) as score
    FROM hudi_vector_search(
        '{TABLE_PATH}', 'embedding', (SELECT q_vec FROM query_input), 5, 'cosine'
    )
""").collect()

print("\U0001F50E Top Semantic Matches:")
for row in matches:
    print(f"  \u2022 {row.title} at {row.company} \u2014 Score: {row.score:.2f}")

The resume never mentions *"Data Scientist"* — but vector search finds them by **meaning**, not keywords.

---
## 5. Hybrid Search — Add Business Constraints

The candidate says: *"I specifically want to work at Reddit."*

We combine the **same vector search** with a standard SQL `WHERE` clause. Vector + SQL in one query.

In [ ]:
# Post-filter approach: fetch a wide vector search window, then apply SQL filters.
# hudi_vector_search() operates on the full vector index and doesn't yet support
# predicate pushdown — so we retrieve broadly and filter after.
# Future optimization: push filters directly into the vector index scan.

hybrid_matches = spark.sql(f"""
    SELECT * FROM (
        SELECT title, company, (1 - _distance) as score
        FROM hudi_vector_search(
            '{TABLE_PATH}', 'embedding', (SELECT q_vec FROM query_input), 3000, 'cosine'
        )
    )
    WHERE company = 'Reddit'
    ORDER BY score DESC
    LIMIT 5
""").collect()

print("\U0001F50E Hybrid Matches (Reddit only):")
for row in hybrid_matches:
    print(f"  \u2022 {row.title} at {row.company} \u2014 Score: {row.score:.2f}")

Same Hudi table. Same vector index. Just added a SQL filter.

---
## 6. Analytics Dashboard

Now we switch hats — we're an analyst on the job platform team. Which companies are hiring the most? What roles dominate the market? We query the **exact same table**. No ETL to a separate warehouse.

In [ ]:
spark.read.format("hudi").load(TABLE_PATH).createOrReplaceTempView("jobs_table")

company_df = spark.sql("""
    SELECT company, count(*) as job_count
    FROM jobs_table GROUP BY company
    ORDER BY job_count DESC LIMIT 15
""").toPandas()

title_df = spark.sql("""
    SELECT title, count(*) as title_count
    FROM jobs_table GROUP BY title
    ORDER BY title_count DESC LIMIT 15
""").toPandas()

fig, axes = plt.subplots(1, 2, figsize=(15, 5))

axes[0].barh(company_df["company"], company_df["job_count"], color="green")
axes[0].set_title("Hiring Activity: Postings by Company")
axes[0].set_xlabel("Number of Postings")
axes[0].invert_yaxis()

axes[1].barh(title_df["title"], title_df["title_count"], color="skyblue")
axes[1].set_title("Most Common Data Science Roles")
axes[1].set_xlabel("Number of Postings")
axes[1].invert_yaxis()

plt.tight_layout()
plt.show()

print("\u2713 Dashboard generated from the same Hudi table.")

---
**One table. Vector search, hybrid search, and analytics. No data copying, no separate vector database.**

In [ ]:
spark.stop()